In [1]:
%pip install stable_baselines3
import os
os.environ["CUDA_VISIBLE_DEVICES"] = ""
from stable_baselines3 import PPO
import gymnasium as gym


gym.register(
    id="RobotObstacleEnv-v0",
    entry_point="robot_obstacles_env:RobotObstacleEnv",
    max_episode_steps=500,
)


[notice] A new release of pip is available: 24.0 -> 25.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
TIMESTEPS = 10000000
env = gym.make("RobotObstacleEnv-v0")

# model = PPO.load("weights/ppo_model")
model = PPO("MultiInputPolicy", env)
model.learn(total_timesteps=TIMESTEPS/1000)

/home/nlin/.pyenv/versions/3.11.11/lib/python3.11/site-packages/gymnasium/envs/registration.py:642: UserWarning: WARN: Overriding environment RobotObstacleEnv-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")


KeyboardInterrupt: 

In [1]:
from util.maze_generator import maze_generator

maze_generator((10, 10))

KeyboardInterrupt: 

In [16]:
obs, _info = env.reset()
for _ in range(1000):
    action, _states = model.predict(obs)
    obs, reward, term, trunc, info = env.step(action)
    done = term or trunc
    if done:
        obs, _info = env.reset()

In [14]:
model.save("weights/ppo_model")